# Chapter 10 시계열을 위한 딥러닝

## 10.2 신경망 프로그래밍

### 10.2.1 데이터, 기호, 연산, 계층, 그래프

기호 A, B를 정의하고, A와 B의 행렬곱 결과를 담은 기호 C를 정의하는 방법

In [1]:
# # 의사코드
# symbol A;
# symbol B;
# symbol C = matmul(A, B);

기호와 데이터의 구분이 필수적임

완전연결계층 by MXnet

In [2]:
!pip install mxnet

In [3]:
import mxnet as mx

In [4]:
fc1 = mx.gluon.nn.Dense(120, activation='relu')

입력, 타깃, 측정하려는 손실의 정의가 추가로 필요하고, 모델이라는 큰 틀 속에<br/>
계층을 배치하는 등의 작업이 필요

In [18]:
# 독립적인 계층을 덩그러니 선언하는 대신 신경망 모델을 만듬
from mxnet.gluon import nn
from mxnet import init
from mxnet import gluon
net = nn.Sequential()
net.add(nn.Dense(120, activation='relu'), nn.Dense(1))
net.initialize(init=init.Xavier())  # 가중치 초기화

# 원하는 손실을 정의
L2Loss = mx.gluon.loss.L2Loss()

trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.01})

학습 에폭 실행

In [19]:
# for data, target in train_data:
#     # 경사를 계산
#     with autograd.record():
#         out = net(data)
#         loss = L2Loss(output, data)
#     loss.backward()
#     # 파라미터 업데이트를 위해 경사를 적용
# trainer.step(batch_size)

모델 및 파라미터 저장

In [20]:
# net.save_parameters('model.params')

## 10.3 학습 파이프라인 만들기

몇 년간 다양한 장소에서 측정된 시간별 전기사용량 데이터

필요한 표준 라이브러리 목록 불러오기

In [23]:
from math import floor

# 저장 모듈
import os

import argparse

# 딥러닝 모듈
import mxnet as mx

# 데이터 처리용 모듈
import numpy as np
import pandas as pd

# 사용자 정의 보고용 모듈
# import perf

In [24]:
# 커맨드라인에서 입력받지 않는 몇가지 하이퍼파라미터가 있음
DATA_SEGMENTS = {'tr': 0.6, 'va': 0.2, 'tst': 0.2}
THRESHOLD_EPOCHS = 5
COR_THRESHOLD = 0.0005

# 파서 설정
parser = argparse.ArgumentParser()

# 데이터의 모양
parser.add_argument('--win', type=int, default=24*7)
parser.add_argument('--h', type=int, default=3)

# 모델의 사양
parser.add_argument('--model', type=str, default='rnn_model')
# cnn의 구성 요소
parser.add_argument('--sz-filt', type=int, default=8)
parser.add_argument('--n-filt', type=int, default=10)
# Rnn의 구성 요소
parser.add_argument('--rnn-units', type=int, default=10)

# 학습의 상세
parser.add_argument('--batch-n', type=int, default=1024)
parser.add_argument('--lr', type=float, default=0.0001)
parser.add_argument('--drop', type=float, default=0.2)
parser.add_argument('--n-epochs', type=int, default=30)

# 작업 내역 저장
parser.add_argument('--data-dir', type=str, default='../data')
parser.add_argument('--save-dir', type=str, default=None)

_StoreAction(option_strings=['--save-dir'], dest='save_dir', nargs=None, const=None, default=None, type=<class 'str'>, choices=None, help=None, metavar=None)

In [27]:
# !pip --drop=0.2 --win=96 --batch-n=128 --lr=0.001 --n-epochs=25
# !pip --data-dir=/data/elec --save-dir=/archive/results
# !pip --model=원하는_모델의_종류